# Utilities

Because types and functions defined in this notebook are imported by many other notebooks, we festidiously avoid side effects. So, we do not print, draw, or set global variables, here.

## *type variables*

Data structures, by definition, are generic containers. To support this genericity, we define type variables here, because Python does not allow type variables to be created on-the-fly in type definitions. In modern functional programming languages with a mathematical bent, like Haskell and Agda, Greek letters are used as type variables, by convention. We shall follow that convention, here.

In [1]:
from typing import TypeVar

α = TypeVar("α")

## *labels*

In algorithms, container data structures are often labelled. So, we define the `Tagged` base class for labelled containers. We do not use `id` in Python, becuase all classes inherit this attribute from the priomodial system class and the runtime uses it to identify objects. This is one of those Python quirks.

In [2]:
Tag = str
class Tagged:
    def __init__(self, tag: Tag):
        self.tag = tag

## *nullability and fallibility*

In functional programming, it is common to use the `Option` type for nullable variables, variables that may not contain any valid data. Let us define this type and the associated utility functions.

In [3]:
from typing import Union

Option = Union[None, α] # same as typing.Optional

def isNone(o: Option[α]) -> bool: return o is None

def isSome(o: Option[α]) -> bool: return not isNone(o)

We also define the `Result` type. In functional programming, we do not throw an exception when a function encounters an error; instead, we return from this fallible function a `Result`, which contains an error or a result value. We will also define a couple of utility functions, as well.

In [4]:
Result = Union[Exception, α]

def isError(r: Result[α]) -> bool: return type(r) is Exception

def isResult(r: Result[α]) -> bool: return not isError(r)

## *infinity*

Many graph algorithms use the $\infty$ to indicate an invalid state. We use the maximum integer value for that purpose.

In [5]:
import sys
Infinity = sys.maxsize

## *disjoint sets*

Disjoint sets are a collection of dynamic sets with no common elements between them. Hence, the disjoint sets and their elements are related by a a bijection: given a disjoint set, we can find a unique element in the entire collection; given an element, we can find the unique disjoint set in the collection that contains it. See Chapter 19 *Data Structures for Disjoint Sets* p.520.

In [6]:
from typing import Callable, Dict, List

class SSet: # sorted set
    def __init__(self, i: List[α], attr: Callable[[float], float]):
        super().__init__()
        self.ii = set(i) # guaranteed to contain at least one item
        self.attr = attr # sorting attribute selector

    def ins(self, i: α) -> None: self.ii = set(sorted([i, *self.ii], key=self.attr))
    def getII(self) -> List[α]: return list(self.ii)
    def getRep(self) -> str: return str(self.getII()[0]) # return the representative item string

    def contains(self, i: α) -> bool: return i in self.ii

class DSet: # disjoint sets (a collection of sorted sets)
    def __init__(self, attr: Callable[[float], float]):
        super().__init__()
        self.ss: Dict[str, SSet] = {} # {str(rep): SSet}
        self.attr = attr

    def getSS(self) -> List[SSet]: return list(self.ss.values())

    def makeSet(self, x: α) -> None:
        s = self.findSet(x)
        if isNone(s): # x is not in the collection
            s = SSet([x], attr=self.attr)
            self.ss[s.getRep()] = s

    def findSet(self, x: α) -> Option[SSet]:
        ss = [s for s in self.getSS() if s.contains(x)]
        return ss[0] if ss != [] else None

    def union(self, x: α, y: α) -> SSet:
        if x == y: return self.findSet(x)
        sx = self.findSet(x)
        sy = self.findSet(y)
        if sx == sy: return sx
        ii = []
        if isSome(sx):
            self.ss.pop(sx.getRep())
            ii += sx.getII()
        if isSome(sy):
            self.ss.pop(sy.getRep())
            ii += sy.getII()
        su = SSet(ii, attr=self.attr)
        self.ss[su.getRep()] = su